Duomenys - gyvuliu sarasas(gyvulio unikalus numeris, gimimo data, paskutinė laikymo data, lytis, apsiversiavimo data ir pan) ir užpildoma Žemės ūkio ministrerijos paruošta ataskaitos forma.

laikotarpis = [('2019-12-31', '2020-01-31'), 
               ('2020-01-31', '2020-02-29'), 
               ('2020-02-29', '2020-03-31'), 
               ('2020-03-31', '2020-04-30'), 
               ('2020-04-30', '2020-05-31'), 
               ('2020-05-31', '2020-06-30'), 
               ('2020-06-30', '2020-07-31'), 
               ('2020-07-31', '2020-08-31'),
               ('2020-08-31', '2020-09-30'), 
               ('2020-07-31', '2020-08-31'), 
               ('2020-08-31', '2020-09-30'), 
               ('2020-09-30', '2020-10-31'), 
               ('2020-10-31', '2020-11-30'), 
               ('2020-11-30', '2020-12-31')]

In [7]:
import openpyxl as xl
import pandas as pd
from datetime import datetime, timedelta

bandos_mazejimas = ('Kritimas', 'Pardavimas')
bandos_didejimas = ('Gimimas', 'Pirkimas')
gyvuliu_grupes = ('Karvė', 'Telyčia iki 2 metų', 'Telyčia virš 2 metų', 
                  'Buliukas iki 2 metų', 'Bulius virš 2 metų', 'Veršeliai')
laikotarpis = [('2019-12-31', '2020-01-31'), 
               ('2020-01-31', '2020-02-29'), 
               ('2020-02-29', '2020-03-31'), 
               ('2020-03-31', '2020-04-30'), 
               ('2020-04-30', '2020-05-31'), 
               ('2020-05-31', '2020-06-30'), 
               ('2020-06-30', '2020-07-31')]

# # Pirma banda:
# duomenys = 'ZL_gyvuliai.xlsx'
# rezultatas = 'Ataskaita_ZL_2020.xlsx'

# Antra banda:
duomenys = 'DZ_gyvuliai.xlsx'
rezultatas = 'Ataskaita_DZ_2020.xlsx'

In [8]:
def darbine_lenta(menesio_pradzia, menesio_pabaiga):
    global duomenys
    gyvuliai = pd.read_excel(duomenys)
    return gyvuliai[((gyvuliai['Laikymo pradžia']) <= menesio_pabaiga + timedelta(1)) 
                     & (gyvuliai['Laikymo pabaiga'] > menesio_pradzia)]

In [9]:
# Buliukas gimsta 36kg, telycaite gimsta 32kg. Prieauglis 1,05kg er dieną.
# Buliaus max svoris 800kg, telycios 600kg

def svorio_skaiciavimas(row):
    svorio_prieaugis_per_diena = 1.05
    if row['Lytis'] == 1:
        gyvulio_svoris = 36 + svorio_prieaugis_per_diena * int(row['Amzius'])
        if gyvulio_svoris > 800:
            return 800
        else:
            return gyvulio_svoris
    else: 
        gyvulio_svoris = 32 + svorio_prieaugis_per_diena * int(row['Amzius'])
        if gyvulio_svoris > 600:
            return 600
        else:
            return gyvulio_svoris

In [10]:
def gyvulio_grupe(row):
    global periodas
    if row['Amzius'] < 365:
        return "Veršeliai"
    elif (row['Amzius'] < 730) and row['Lytis'] == 1:
        return "Buliukas iki 2 metų"
    elif row['Lytis'] == 1:
        return "Bulius virš 2 metų"
    elif (row['Gimimo data'] < row['Apsiveršiavimo data'] < periodas):
        return "Karvė"
    elif row['Amzius'] < 730:
        return "Telyčia iki 2 metų"
    else:
        return 'Telyčia virš 2 metų'

In [11]:
def amzius(row):
    global periodas
    if row['Laikymo pabaiga'] < periodas:
        return row['Laikymo pabaiga'] - row['Gimimo data']
    else:
        return periodas - row['Gimimo data']

In [12]:
wb = xl.load_workbook(rezultatas)
sheet = wb['Sheet1']

rows, column = 6, 2

for periodai in laikotarpis:
    menesio_pradzia, menesio_pabaiga =  periodai
    menesio_pradzia = datetime.strptime(menesio_pradzia, '%Y-%m-%d')
    menesio_pabaiga = datetime.strptime(menesio_pabaiga, '%Y-%m-%d')
    calc_period  = darbine_lenta(menesio_pradzia, menesio_pabaiga)
    
    periodo_pabaiga = False
    for periodas in periodai:
        
        #Skaičiuojamas gyvulio amzius, nuo kurio priklauso grupe ir svoris
        #Skaičiuojamas gyvulio svoris
        #Gyvuliu suskirstymas į grupės
        periodas = datetime.strptime(periodas, '%Y-%m-%d')
        calc_period['Amzius'] = calc_period.apply(amzius, axis = 1)
        calc_period['Amzius'] = calc_period.Amzius.apply(lambda x: x.days)             
        calc_period['Svoris'] = calc_period.apply(svorio_skaiciavimas, axis = 1)        
        calc_period['Gyvulio grupe'] = calc_period.apply(gyvulio_grupe, axis = 1)
         
        row = rows
        for grupe in gyvuliu_grupes:
            svoris = calc_period[(calc_period['Gyvulio grupe'] == grupe) 
                                 & (calc_period['Laikymo pabaiga'] >= periodas)
                                 & (calc_period['Gimimo data'] <= periodas)]['Svoris'].sum()
            kiekis = calc_period[(calc_period['Gyvulio grupe'] == grupe) 
                                 & (calc_period['Laikymo pabaiga'] >= periodas)
                                 & (calc_period['Gimimo data'] <= periodas)]['Svoris'].count()
            cell = sheet.cell(row = row, column = column)
            cell.value = kiekis
            cell = sheet.cell(row = row, column = column + 1)
            cell.value = svoris
            
            if periodo_pabaiga:
               
                #Didėjimo ataskaitos
                column_kintamasis = 0
                for didejimas in bandos_didejimas:
                    gimimai_svoris = calc_period[(calc_period['Laikymo pradžia'].dt.month == periodas.month)
                                                 & (calc_period['Laikymo pradžia'].dt.year == periodas.year)
                                                 & (calc_period['Budas_isigijimas'] == didejimas)
                                                 & (calc_period['Gyvulio grupe'] == grupe)]['Svoris'].sum()
                    if gimimai_svoris> 0:
                        cell = sheet.cell(row = row, column = column - 17 + column_kintamasis)
                        cell.value = gimimai_svoris                             
                        gimimai_kiekis = calc_period[(calc_period['Laikymo pradžia'].dt.month == periodas.month)
                                                     & (calc_period['Laikymo pradžia'].dt.year == periodas.year)
                                                     & (calc_period['Budas_isigijimas'] == didejimas)
                                                     & (calc_period['Gyvulio grupe'] == grupe)]['Svoris'].count()
                        cell = sheet.cell(row = row, column = column- 18 + column_kintamasis)
                        cell.value = gimimai_kiekis
                    column_kintamasis = 2
                
                #Praradimo ataskaitos
                column_kintamasis = 7
                for mazejimas in bandos_mazejimas:
                    parduotas_svoris = calc_period[(calc_period['Laikymo pabaiga'].dt.month == periodas.month)
                                                 & (calc_period['Laikymo pabaiga'].dt.year == periodas.year)
                                                & (calc_period['Budas_praradimas'] == mazejimas)
                                                & (calc_period['Gyvulio grupe'] == grupe)]['Svoris'].sum()
                    if parduotas_svoris> 0:
                        cell = sheet.cell(row = row, column = column - column_kintamasis)
                        cell.value = parduotas_svoris                             
                        parduotas_kiekis = calc_period[(calc_period['Laikymo pabaiga'].dt.month == periodas.month)
                                                 & (calc_period['Laikymo pabaiga'].dt.year == periodas.year)
                                                & (calc_period['Budas_praradimas'] == mazejimas)
                                                & (calc_period['Gyvulio grupe'] == grupe)]['Svoris'].count()
                        cell = sheet.cell(row = row, column = column - column_kintamasis - 1)
                        cell.value = parduotas_kiekis
                    column_kintamasis = 5
                
                lyginimo_lentele = pd.merge(per_pradzia, calc_period, on = 'Numeris')
                lyginimo_lentele = lyginimo_lentele[['Numeris', 'Gyvulio grupe_x', 'Gyvulio grupe_y', 'Svoris_y']]
                lyginimo_lentele = lyginimo_lentele[lyginimo_lentele['Gyvulio grupe_x'] != lyginimo_lentele['Gyvulio grupe_y']]
                
                svoris = lyginimo_lentele[(lyginimo_lentele['Gyvulio grupe_x'] == grupe)]['Svoris_y'].sum()
                cell = sheet.cell(row = row, column = column - 11)
                if svoris > 0:
                    cell.value = svoris
                    kiekis = lyginimo_lentele[(lyginimo_lentele['Gyvulio grupe_x'] == grupe)]['Svoris_y'].count()
                    cell = sheet.cell(row = row, column = column - 12)
                    cell.value = kiekis 

                svoris = lyginimo_lentele[(lyginimo_lentele['Gyvulio grupe_y'] == grupe)]['Svoris_y'].sum()
                cell = sheet.cell(row = row, column = column - 9)
                if svoris > 0:
                    cell.value = svoris
                    kiekis = lyginimo_lentele[(lyginimo_lentele['Gyvulio grupe_y'] == grupe)]['Svoris_y'].count()
                    cell = sheet.cell(row = row, column = column - 10)
                    cell.value = kiekis 
                    

                
            row += 1

        column = 22
        per_pradzia = calc_period.copy()
        periodo_pabaiga = True
        
    rows += 13
    column = 2
    
wb.save(rezultatas)